In [1]:
# do these need to be rounded?

In [2]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [3]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [4]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [5]:
taz_900 = r'E:\Projects\REMM-Manage-Base-Year-Data\Ancillary\TAZ.gdb\TAZ_900_withSmallArea'
# city_900 = r'E:\Projects\REMM-Manage-Base-Year-Data\Ancillary\TAZ.gdb\CITYAREA_900'
parcels = r'E:\Projects\REMM-Manage-Base-Year-Data\Current_Inputs\remm_base_year.gdb\parcels'
parcel_pts = arcpy.FeatureToPoint_management(parcels, os.path.join(gdb, 'parcel_pts'), "INSIDE")
parcel_eq = pd.read_csv(r"E:\parcel_eq_v7.csv")

In [6]:
# # update parcel eq
# small_area = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\WFRC_Dashboard\Outputs\parcel_small_area")[['parcel_id', 'NewSA']].copy()
# small_area.rename({'NewSA':'SMALLAREA'}, axis=1, inplace=True)
# parcel_eq = parcel_eq.merge(small_area, on='parcel_id', how='left')
# parcel_eq.to_csv(r"E:\parcel_eq_v7.csv") 

In [7]:
transit_walkshed_2019 = r".\Inputs\Walksheds\Walksheds.gdb\transit_walkshed_2019"
transit_walkshed_2028 = r".\Inputs\Walksheds\Walksheds.gdb\transit_walkshed_2028"
transit_walkshed_2032 = r".\Inputs\Walksheds\Walksheds.gdb\transit_walkshed_2032"
transit_walkshed_2042 = r".\Inputs\Walksheds\Walksheds.gdb\transit_walkshed_2042"
transit_walkshed_2050 = r".\Inputs\Walksheds\Walksheds.gdb\transit_walkshed_2050"
parks_walkshed = r".\Inputs\Walksheds\Walksheds.gdb\parks_walkshed"
trails_walkshed = r".\Inputs\Walksheds\Walksheds.gdb\trails_walkshed"

In [8]:
# prep the taz table
taz_df = pd.DataFrame.spatial.from_featureclass(taz_900)[['SA_TAZID', 'CO_TAZID', 'CO_FIPS', 'CO_NAME', 'SMALLAREA', 'CityArea','DEVACRES', 'SHAPE']].copy()
taz_df.rename({'SA_TAZID':'TAZID_900', 'CityArea':'CITYAREA'}, axis=1, inplace=True)
transit_taz_df = taz_df.copy()
parks_taz_df = taz_df.copy()
trailheads_taz_df = taz_df.copy()
centers_taz_df = taz_df.copy()

# # prep the city area table
# city_df = pd.DataFrame.spatial.from_featureclass(city_900)[['CityArea','DEVACRES', 'SHAPE']].copy()
# city_df.rename({'CityArea':'CityArea'}, axis=1, inplace=True)
# transit_city_df = city_df.copy()
# parks_city_df = city_df.copy()
# trailheads_city_df = city_df.copy()
# centers_city_df = city_df.copy()

for year in range(2019, 2051):
    if (year % 2 ==0) or (year == 2019): print(year)
    short_year = str(year)[2:]
    
    ###########
    # Transit
    ###########

    # use the year to select the walkshed
    if (year < 2028): transit_walkshed = transit_walkshed_2019
    if (year >= 2028) and (year < 2032): transit_walkshed = transit_walkshed_2028
    if (year >= 2032) and (year < 2042): transit_walkshed = transit_walkshed_2032
    if (year >= 2042) and (year < 2050): transit_walkshed = transit_walkshed_2042
    if (year >= 2050): transit_walkshed = transit_walkshed_2050

    # create a layer and filter for the 20 minute polygon 
    transit_walkshed_lyr =  arcpy.MakeFeatureLayer_management(transit_walkshed, 'transit_walkshed_lyr', where_clause=""" ToBreak = 20 """) 

    # spatial join to walkshed areas
    target_features = parcel_pts 
    join_features = transit_walkshed_lyr
    output_features = os.path.join(gdb, "pts_transit_walkshed_sj")

    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(target_features)
    fieldmappings.addTable(join_features)

    transit_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, match_option="INTERSECT")

    transit_sj_df = pd.DataFrame.spatial.from_featureclass(transit_sj[0])[['parcel_id','ToBreak']].copy()
    transit_sj_df.loc[transit_sj_df['ToBreak'] == 20, 'transit'] = 1
    transit_sj_df = transit_sj_df[['parcel_id','transit']].copy()

    ###########
    # Parks
    ###########
    
    # create a layer and filter for the 20 minute polygon 
    parks_walkshed_lyr =  arcpy.MakeFeatureLayer_management(parks_walkshed, 'parks_walkshed_lyr', where_clause=""" ToBreak = 20 """) 

    # spatial join to walkshed areas
    target_features = parcel_pts 
    join_features = parks_walkshed_lyr
    output_features = os.path.join(gdb, "pts_parks_walkshed_sj")

    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(target_features)
    fieldmappings.addTable(join_features)

    parks_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, match_option="INTERSECT")

    parks_sj_df = pd.DataFrame.spatial.from_featureclass(parks_sj[0])[['parcel_id','ToBreak']].copy()
    parks_sj_df.loc[parks_sj_df['ToBreak'] == 20, 'parks'] = 1
    parks_sj_df = parks_sj_df[['parcel_id','parks']].copy()

    #############
    # Trailheads
    #############

    # create a layer and filter for the 20 minute polygon 
    trails_walkshed_lyr =  arcpy.MakeFeatureLayer_management(trails_walkshed, 'trails_walkshed_lyr', where_clause=""" ToBreak = 20 """) 

    # spatial join to walkshed areas
    target_features = parcel_pts 
    join_features = trails_walkshed_lyr
    output_features = os.path.join(gdb, "pts_trailheads_walkshed_sj")

    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(target_features)
    fieldmappings.addTable(join_features)

    trails_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, match_option="INTERSECT")

    trails_sj_df = pd.DataFrame.spatial.from_featureclass(trails_sj[0])[['parcel_id','ToBreak']].copy()
    trails_sj_df.loc[trails_sj_df['ToBreak'] == 20, 'trailheads'] = 1
    trails_sj_df = trails_sj_df[['parcel_id','trailheads']].copy()

    ##################
    # merge data
    ##################

    # read in se data
    parsel_se = pd.read_pickle(r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\Parcel_SE_6Runs_Average_20230613\averaged_parcel_se_{}.pkl".format(year))

    # parcel_eq.rename({"CITY_AREA": "CityArea"}, axis=1, inplace=True)
    parsel_se = parsel_se.merge(parcel_eq, on='parcel_id', how='left')
    
    parsel_se = parsel_se.merge(transit_sj_df, on ='parcel_id', how='left')
    parsel_se = parsel_se.merge(parks_sj_df, on ='parcel_id', how='left')
    parsel_se = parsel_se.merge(trails_sj_df, on ='parcel_id', how='left')

    # transit
    parcel_se_transit = parsel_se[parsel_se['transit']==1].copy()
    taz_transit = parcel_se_transit.groupby('TAZID_900', as_index=False)[['households']].sum()
    taz_transit.columns = ['TAZID_900', f'HH_{year}']
    # city_transit = parcel_se_transit.groupby('CityArea', as_index=False)[['households']].sum()
    # city_transit.columns = ['CityArea', f'YEAR{short_year}']

    # parks
    parcel_se_parks = parsel_se[parsel_se['parks']==1].copy()
    taz_parks = parcel_se_parks.groupby('TAZID_900', as_index=False)[['households']].sum()
    taz_parks.columns = ['TAZID_900', f'HH_{year}']
    # city_parks = parcel_se_parks.groupby('CityArea', as_index=False)[['households']].sum()
    # city_parks.columns = ['CityArea', f'YEAR{short_year}']

    # trailheads
    parcel_se_trailheads = parsel_se[parsel_se['trailheads']==1].copy()
    taz_trailheads = parcel_se_trailheads.groupby('TAZID_900', as_index=False)[['households']].sum()
    taz_trailheads.columns = ['TAZID_900', f'HH_{year}']
    # city_trailheads = parcel_se_trailheads.groupby('CityArea', as_index=False)[['households']].sum()
    # city_trailheads.columns = ['CityArea', f'YEAR{short_year}']

    # centers
    parcel_se_centers = parsel_se[parsel_se['CENTER_NAME']!= "Non-center"].copy()
    taz_centers = parcel_se_centers.groupby('TAZID_900', as_index=False)[['hhpop']].sum()
    taz_centers.columns = ['TAZID_900', f'POP_{year}']
    # city_centers = parcel_se_centers.groupby('CityArea', as_index=False)[['hhpop']].sum()
    # city_centers.columns = ['CityArea', f'YEAR{short_year}']

    # merge back to taz geometry
    transit_taz_df = transit_taz_df.merge(taz_transit, on='TAZID_900', how='left')
    parks_taz_df = parks_taz_df.merge(taz_parks, on='TAZID_900', how='left')
    trailheads_taz_df = trailheads_taz_df.merge(taz_trailheads, on='TAZID_900', how='left')
    centers_taz_df = centers_taz_df.merge(taz_centers, on='TAZID_900', how='left')

    # # merge back to city area geometry
    # transit_city_df = transit_city_df.merge(city_transit, on='CityArea', how='left')
    # parks_city_df = parks_city_df.merge(city_parks, on='CityArea', how='left')
    # trailheads_city_df = trailheads_city_df.merge(city_trailheads, on='CityArea', how='left')
    # centers_city_df = centers_city_df.merge(city_centers, on='CityArea', how='left')

    

2019
2020
2022
2024
2026
2028
2030
2032
2034
2036
2038
2040
2042
2044
2046
2048
2050


In [9]:
transit_taz_df = fill_na_sedf(transit_taz_df)
parks_taz_df = fill_na_sedf(parks_taz_df)
trailheads_taz_df = fill_na_sedf(trailheads_taz_df)
centers_taz_df = fill_na_sedf(centers_taz_df)

transit_taz_df.spatial.to_featureclass(location=os.path.join(gdb2,'households_within_20min_walk_to_transit_by_taz'),sanitize_columns=False)
parks_taz_df.spatial.to_featureclass(location=os.path.join(gdb2,'households_within_20min_walk_to_parks_by_taz'),sanitize_columns=False)
trailheads_taz_df.spatial.to_featureclass(location=os.path.join(gdb2,'households_within_20min_walk_to_trailheads_by_taz'),sanitize_columns=False)
centers_taz_df.spatial.to_featureclass(location=os.path.join(gdb2,'population_within_centers_by_taz'),sanitize_columns=False)

# transit_city_df = fill_na_sedf(transit_city_df)
# parks_city_df = fill_na_sedf(parks_city_df)
# trailheads_city_df = fill_na_sedf(trailheads_city_df)
# centers_city_df = fill_na_sedf(centers_city_df)

# transit_city_df.spatial.to_featureclass(location=os.path.join(gdb2,'households_within_20min_walk_to_transit_by_city'),sanitize_columns=False)
# parks_city_df.spatial.to_featureclass(location=os.path.join(gdb2,'households_within_20min_walk_to_parks_by_city'),sanitize_columns=False)
# trailheads_city_df.spatial.to_featureclass(location=os.path.join(gdb2,'households_within_20min_walk_to_trailheads_by_city'),sanitize_columns=False)
# centers_city_df.spatial.to_featureclass(location=os.path.join(gdb2,'population_within_centers_by_city'),sanitize_columns=False)

'e:\\Tasks\\WFRC_Dashboard\\Outputs\\results.gdb\\population_within_centers_by_taz'